In [31]:
import pandas as pd
from collections import Counter
import statistics
import category_encoders as ce 

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')

import time

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/maxmercury/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [28]:
hotels = pd.read_csv('https://firebasestorage.googleapis.com/v0/b/civic-abode-211408.appspot.com/o/hotels.csv?alt=media&token=aa9c8eac-6a57-4caa-bfa4-e5a3b198d2fd')
hotels.head(2) 

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097


In [32]:
hotels['lat'] = hotels['lat'].fillna(statistics.median(hotels['lat']))
hotels['lng'] = hotels['lat'].fillna(statistics.median(hotels['lat']))

hotels['days_since_review'] = hotels['days_since_review'].apply(lambda x: x.split(' ')[0])
hotels['days_since_review'] = hotels['days_since_review'].astype(int)
hotels['review_date'] = pd.to_datetime(hotels['review_date'])
hotels['review_month'] = hotels['review_date'].dt.month

hotels['hotel_address'][11].split(' ')[-2:]
hotels['hotel_country'] = hotels['hotel_address'].apply(lambda x: ' '.join(x.split(' ')[-2:]) if 'United Kingdom' in x else x.split(' ')[-1])

In [33]:
bin_encoder = ce.BinaryEncoder(cols=['hotel_name', 'reviewer_nationality'])
hotels_bin = bin_encoder.fit_transform(hotels[['hotel_name', 'reviewer_nationality']])
hotels = pd.concat([hotels, hotels_bin], axis=1)

encoder = ce.OneHotEncoder(cols=['hotel_country'])
country_bin = encoder.fit_transform(hotels['hotel_country'])
hotels = pd.concat([hotels, country_bin], axis=1)

In [34]:
hotels['tags'][0][3:-3].split(" ', ' ")

tags_dict = {}
def get_tags_count(tag):
    tag = tag[3:-3].split(" ', ' ")
    for el in tag:
        if el in tags_dict:
            tags_dict[el] += 1
        else:
            tags_dict[el] = 1
    return

hotels['tags'].apply(get_tags_count)
tags_df = pd.DataFrame(tags_dict, index=['count']).transpose()
top_tags = list(tags_df['count'].sort_values(ascending=False).index[:20])

for tag in top_tags:
    hotels[tag] = hotels['tags'].apply(lambda x: 1 if tag in x else 0)

In [35]:
sent_analyzer = SentimentIntensityAnalyzer()

neg_reviews = hotels['negative_review'].apply(lambda x: sent_analyzer.polarity_scores(x))
pos_reviews = hotels['positive_review'].apply(lambda x: sent_analyzer.polarity_scores(x))
hotels.loc[:,['n_negative', 'n_neutral', 'n_positive', 'n_compound']] = list(neg_reviews.apply(lambda x: [x['neg'], x['neu'], x['pos'], x['compound']]).values)

In [36]:
hotels.drop(['hotel_address', 'hotel_name', 'reviewer_nationality', 'hotel_country', 'tags', 'review_date', 'negative_review', 'positive_review'], axis=1, inplace=True)

In [38]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
X = hotels.drop(['reviewer_score'], axis = 1)  
y = hotels['reviewer_score'] 

In [39]:
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  

In [40]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [41]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
      
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
      
# Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  


In [42]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))

MAPE: 0.13162514511794274


Небольшой бонус:


In [10]:
# # убираем признаки которые еще не успели обработать, 
# # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
# object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
# hotels.drop(object_columns, axis = 1, inplace=True)

# # заполняем пропуски самым простым способом
# hotels = hotels.fillna(0)